# Feature Extraction: Open Street Map Data

In [1]:
import geopandas as gpd
import pandas as pd 
import osmnx as ox 
import numpy as np

/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/gp/knk8fb1937d_5p9ybs31txzr0000gn/T/ipykernel_10149/59577225.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged 

## Read the OSM data for Köln

In [48]:
neighborhoods = gpd.read_file("../data/raw/3 Neighborhoods/Neighborhoods_Köln.gpkg").to_crs(epsg = 4326)
neighborhoods.Neighborhood_Name.unique()


array(['Porz', 'Junkersdorf', 'Widdersdorf', 'Stammheim', 'Höhenhaus',
       'Raderberg', 'Bayenthal', 'Eil', 'Grengel', 'Mauenheim',
       'Ossendorf', 'Vogelsang', 'Nippes', 'Holweide', 'Bilderstöckchen',
       'Merheim', 'Ostheim', 'Dünnwald', 'Bocklemünd/Mengenich', 'Vingst',
       'Rath/Heumar', 'Neustadt/Süd', 'Müngersdorf', 'Altstadt/Nord',
       'Brück', 'Niehl', 'Roggendorf/Thenhoven', 'Flittard', 'Dellbrück',
       'Altstadt/Süd', 'Neustadt/Nord', 'Lindweiler', 'Braunsfeld',
       'Neuehrenfeld', 'Esch/Auweiler', 'Sülz', 'Lindenthal', 'Ehrenfeld',
       'Mülheim', 'Riehl', 'Lövenich', 'Weiß', 'Buchheim', 'Langel',
       'Höhenberg', 'Neubrück', 'Volkhoven/Weiler', 'Chorweiler', 'Deutz',
       'Godorf', 'Elsdorf', 'Zollstock', 'Sürth', 'Immendorf',
       'Bickendorf', 'Meschenich', 'Longerich', 'Rondorf', 'Weiden',
       'Lind', 'Finkenberg', 'Ensen', 'Hahnwald', 'Seeberg', 'Raderthal',
       'Wahn', 'Merkenich', 'Klettenberg', 'Rodenkirchen', 'Westhoven',
       

In [60]:
# e.g. restaurants in all of Köln 
nb = neighborhoods.loc[neighborhoods.Neighborhood_Name=='Neustadt/Süd']
restaurants = ox.geometries.geometries_from_polygon(polygon=nb.geometry.iloc[0], tags={"amenity":"restaurant"})
print(restaurants.shape)

(159, 92)


/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/set_operations.py:340: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)


#### Store amenities in a list for each neighborhood 



In [67]:
# Define the tag to extract (amenity=restaurant)
tag = {'amenity':True}

# Define an empty list to store the results
results = {}

# Loop over each neighborhood and extract the restaurants
for i, nb_name in enumerate(neighborhoods.Neighborhood_Name):
    nb = neighborhoods.loc[neighborhoods.Neighborhood_Name == nb_name]
    restaurants = ox.geometries.geometries_from_polygon(polygon=nb.geometry.iloc[0], tags=tag)
    # print(f'Processed {i+1}/{len(neighborhoods)} neighborhoods ({nb_name}): found {restaurants.shape[0]} restaurants')
    results[nb_name] = restaurants


/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/set_operations.py:340: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/set_operations.py:340: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shape

In [87]:
nb_results = []

for i, nb_name in enumerate(neighborhoods.Neighborhood_Name):
    nb_result = pd.DataFrame(results[nb_name].amenity.value_counts().to_dict(), index=[i])
    nb_result["Neighborhood_Name"]=nb_name
    nb_results.append(nb_result)

# Combine the results into a single GeoDataFrame
combined_results = pd.concat(nb_results, ignore_index=True)

0                  Porz
1           Junkersdorf
2           Widdersdorf
3             Stammheim
4             Höhenhaus
            ...        
81            Fühlingen
82           Blumenberg
83                Libur
84                 Poll
85    Humboldt/Gremberg
Name: Neighborhood_Name, Length: 86, dtype: object

### Merge with main dataframe 

In [89]:
# read the data
merged_data = gpd.read_file("../data/interim/merged_data.gpkg")
merged_data = merged_data.merge(combined_results, on = "Neighborhood_Name")

In [95]:
merged_data.to_file('../data/interim/merged_data_amenities.gpkg', driver='GPKG')